# Homework # 3 - Logistic Regression - Hung Quoc Tran ID 23856431
Data file: breast_cancer_diagnosis.csv

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Load data

In [2]:
! pwd

/Users/rictq/Documents/CIS_3920_Homework


In [3]:
! head "data/breast_cancer_diagnosis.csv"

In [4]:
# Read data from file into dataframe
df = pd.read_csv('data/breast_cancer_diagnosis.csv')

## Examine data

In [5]:
df.head()

,id,name,radius,texture,perimeter,area,smoothness,compactness,concavity,symmetry,fractal_dimension,age,diagnosis
0,ID842302,Glynnis Munson,NaN,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.2419,0.07871,35,1
1,ID842517,Lana Behrer,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.1812,0.05667,27,1
2,ID84300903,Devondra Vanvalkenburgh,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.2069,0.05999,31,1
3,ID84348301,Glory Maravalle,NaN,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.2597,0.09744,49,1
4,ID84358402,Mellie Mccurdy,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.1809,0.05883,20,1


In [6]:
df.shape

(569, 13)

In [7]:
df.describe()

,radius,texture,perimeter,area,smoothness,compactness,concavity,symmetry,fractal_dimension,age,diagnosis
count,498.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.326635,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.181162,0.062798,39.467487,0.372583
std,3.506881,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.027414,0.007060,13.604683,0.483918
min,7.760000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.106000,0.049960,16.000000,0.000000
25%,11.817500,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.161900,0.057700,27.000000,0.000000
50%,13.465000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.179200,0.061540,40.000000,0.000000
75%,16.155000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.195700,0.066120,52.000000,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.304000,0.097440,62.000000,1.000000


#### Note: NaN in radius column

## Prepare data

### Check for missing values

In [8]:
df.isnull().sum()

id                    0
name                  0
radius               71
texture               0
perimeter             0
area                  0
smoothness            0
compactness           0
concavity             0
symmetry              0
fractal_dimension     0
age                   0
diagnosis             0
dtype: int64

### Handle missing values 

In [9]:
# Drop all rows where radius is NaN
df = df.dropna(subset=['radius'])
df.isnull().sum()

id                   0
name                 0
radius               0
texture              0
perimeter            0
area                 0
smoothness           0
compactness          0
concavity            0
symmetry             0
fractal_dimension    0
age                  0
diagnosis            0
dtype: int64

In [10]:
# Drop non-numeric variables
# Remember to use "inplace=True"
# df.dtypes
non_numeric = df.select_dtypes(include=['object'])
df.drop(non_numeric, axis=1, inplace=True)

In [11]:
df

,radius,texture,perimeter,area,smoothness,compactness,concavity,symmetry,fractal_dimension,age,diagnosis
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.1812,0.05667,27,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.2069,0.05999,31,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.1809,0.05883,20,1
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.1794,0.05742,38,1
10,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.1528,0.05697,51,1
...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.1726,0.05623,18,1
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.1752,0.05533,24,1
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.1590,0.05648,21,1
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.2397,0.07016,41,1


### Separate independent and dependent variables
* Independent variables: All remaining variables except Diagnosis
* Dependent variable: Diagnosis

In [12]:
# Prepare dataset for model training
x = df.iloc[:,:-1].values # all but last column dianogsis
y = df.iloc[:, -1] # diagnosis is the last column

### Split data into training and test sets

In [13]:
# Split data into training data (70%) and test data (30%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## Train model

In [14]:
# Train model
model = LogisticRegression()
model.fit(x_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### If above results in error, review error message, look up documentation for LogisticRegression, and change model parameter appropriately

In [15]:
# Train model again
model = LogisticRegression(max_iter = 200)
model.fit(x_train, y_train)

LogisticRegression(max_iter=200)

## Test model

In [16]:
# Print predictions
predictions = model.predict(x_test)
print(predictions)

[0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 0
 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0
 1 1]


## Model evaluation

In [17]:
# Print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96        93
           1       0.92      0.95      0.93        57

    accuracy                           0.95       150
   macro avg       0.94      0.95      0.94       150
weighted avg       0.95      0.95      0.95       150



### Model accuracy

In [18]:
# Print model accuracy
accuracy = model.score(x_test, y_test)
print("accuracy =", accuracy * 100, "%")

accuracy = 94.66666666666667 %
